In [9]:
from random import randint, choices
import numpy as np

In [18]:
corpus = [
    ["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]
]

K = 4

In [19]:
alpha = .1
beta = .1

In [20]:
theta = dict() # 문서-토픽 분포(빈도) => m번째 문서, k번째 토픽 => {m:{k:빈도}}
phi = dict() # 단어픽 분포(빈도) => m번째 문서, k번째 토픽 => {m:{k:빈도}}
z = dict() # 문서-단어 토픽 할당 정보 => m번째 문서, l번째 단어 => {m:{l:k}}

V = list()
for m, d in enumerate(corpus):
    for l, w in enumerate(d):
        V.append(w.lower())
        if m not in z:
            z[m] = dict()
        if l not in z[m]:
            z[m][l] = 0
        z[m][l] = randint(1,K) # 토픽 할당

V = set(V)
i2w = {i: v for i, v in enumerate(V)}
w2i = {v: i for i, v in enumerate(V)}

for m, L in z.items():
    if m not in theta:
        theta[m] = dict()
    for l, t in L.items():
        if t not in theta[m]:
            theta[m][t] = 0
        theta[m][t] += 1

        v = w2i[corpus[m][l].lower()]

        if v not in phi:
            phi[v] = dict()
        if t not in phi[v]:
            phi[v][t] = 0
        phi[v][t] += 1

In [23]:
iteration = 100

for _ in range(iteration):
    for m, d in enumerate(corpus):
        for l, w in enumerate(d):
            v = w2i[w.lower()]

            k = z[m][l] # gibb's 대상 삭제
            theta[m][k] -= 1
            phi[v][k] -= 1

            topic = list()
            for t in range(K):
                A = (phi[v].get(t+1,0) + beta) / sum([T.get(t+1,0)+beta for v, T in phi.items()])
                B = theta[m].get(t+1,0) + alpha
                topic.append(A*B)

            k = choices(range(K), weights=topic, k=1)[0]+1

            z[m][l] = k
            theta[m][k] = (theta[m][k] if k in theta[m] else 0) + 1
            phi[v][k] = (phi[v][k] if k in phi[v] else 0) + 1

In [32]:
for l, T in phi.items():
    if T[2] > 0:
        print(i2w[l], T[2])

java 1
python 1
r 2
statistics 2
regression 3
probability 1
decision trees 1
libsvm 2
c++ 2
haskell 1
programming languages 1
mathematics 1
support vector machines 1


[1] => ?
7/17 => hadoop, big data, storm, mapreduce
5/17 => java, r, c++
5/17 => probability,
[2] => AI
0/15 =>
1/15 => java
14/15 => AI
[3] => big data
[4] => Data Analysis